In [46]:
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from panther.nn import SKConv2d

device = torch.device("cpu")
device = torch.device("cuda")


class CNN(nn.Module):
    def __init__(self, use_custom=False):
        super(CNN, self).__init__()
        self.conv1 = (
            SKConv2d(3, 16, kernel_size=3, stride=1, padding=1, num_terms=1, low_rank=4)
            if use_custom
            else nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        )

        self.fc1 = nn.Linear(16 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNN(nn.Module):
    def __init__(self, use_custom=False):
        super(CNN, self).__init__()
        self.conv1 = (
            SKConv2d(
                40, 80, kernel_size=3, stride=1, padding=1, num_terms=2, low_rank=15
            )
            if use_custom
            else nn.Conv2d(40, 80, kernel_size=3, stride=1, padding=1)
        )

    def forward(self, x):
        return self.conv1(x)

In [ ]:
def get_dataloader(batch_size=64):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = torchvision.datasets.CIFAR10(
        root="./data", train=True, download=True, transform=transform
    )
    trainloader = DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=2
    )
    testset = torchvision.datasets.CIFAR10(
        root="./data", train=False, download=True, transform=transform
    )
    testloader = DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=2
    )
    return trainloader, testloader


def time_forward_pass(model, input_tensor, warmup=10, repeat=100):
    model.to(device)
    input_tensor = input_tensor.to(device)
    model.eval()  # no dropout/batchnorm updates

    # Warm-up (especially important for CUDA)
    with torch.no_grad():
        for _ in range(warmup):
            _ = model(input_tensor)

    # Actual timing
    torch.cuda.synchronize() if device.type == "cuda" else None
    start = time.time()

    with torch.no_grad():
        for _ in range(repeat):
            _ = model(input_tensor)

    torch.cuda.synchronize() if device.type == "cuda" else None
    end = time.time()

    avg_time = (end - start) / repeat

    return avg_time


# Training function
def train_model(model, trainloader, epochs=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}")


# Compare custom vs PyTorch Conv2D
def compare_models():
    trainloader, testloader = get_dataloader()

    # Train model with PyTorch Conv2D
    print("Training PyTorch Conv2D model...")
    pytorch_model = CNN(use_custom=False)
    # train_model(pytorch_model, trainloader, device)
    t = time_forward_pass(
        pytorch_model, torch.randn(10, 40, 32, 32).to(device), warmup=10, repeat=100
    )
    print(f"PyTorch Conv2D forward pass time: {t:.6f} seconds")
    # Train model with Custom Conv2D
    print("Training Custom Conv2D model...")
    custom_model = CNN(use_custom=True)
    # train_model(custom_model, trainloader, device)
    t = time_forward_pass(
        custom_model, torch.randn(10, 40, 32, 32).to(device), warmup=10, repeat=100
    )
    print(f"Custom Conv2D forward pass time: {t:.6f} seconds")


In [48]:
compare_models()
# 1:26 m:sec for normal conv2d
# 1:38 m:sec for custom conv2d

Files already downloaded and verified
Files already downloaded and verified
Training PyTorch Conv2D model...
PyTorch Conv2D forward pass time: 0.002767 seconds
Training Custom Conv2D model...
Custom Conv2D forward pass time: 0.001522 seconds
